# Modélisation #2 RandomForest + HyperOpt

In [1]:
# Importation des packages

# Base
import importlib
import functions
importlib.reload(functions)
from functions import *

# Data Management
import pandas as pd
import numpy as np

# Modélisation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Optimisation
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Users/hugopuybareau/Documents/GitHub/QRT_DATA_CHALLENGE/env_qrt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Importation des données

train_data = pd.read_csv('../data/Train_Data/3_train_data.csv', index_col=0)
train_scores = pd.read_csv('../data/Y_train_1rknArQ.csv', index_col=0)

sub_data = pd.read_csv('../data/Sub_Data/3_sub_data.csv', index_col=0)

train_data.head(5)

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,HOME_TEAM_YELLOWCARDS_season_sum,HOME_TEAM_REDCARDS_season_sum,HOME_TEAM_OFFSIDES_season_sum,HOME_TEAM_ATTACKS_season_sum,HOME_TEAM_PENALTIES_season_sum,HOME_TEAM_SUBSTITUTIONS_season_sum,HOME_TEAM_BALL_SAFE_season_sum,HOME_TEAM_DANGEROUS_ATTACKS_season_sum,HOME_TEAM_INJURIES_season_sum,HOME_TEAM_GOALS_season_sum,HOME_TEAM_GAME_WON_season_sum,HOME_TEAM_GAME_DRAW_season_sum,HOME_TEAM_GAME_LOST_season_sum,HOME_TEAM_SHOTS_TOTAL_season_average,HOME_TEAM_SHOTS_INSIDEBOX_season_average,HOME_TEAM_SHOTS_OFF_TARGET_season_average,HOME_TEAM_SHOTS_ON_TARGET_season_average,HOME_TEAM_SHOTS_OUTSIDEBOX_season_average,HOME_TEAM_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average,HOME_TEAM_SAVES_season_average,HOME_TEAM_CORNERS_season_average,HOME_TEAM_BALL_POSSESSION_season_average,HOME_TEAM_FOULS_season_average,HOME_TEAM_YELLOWCARDS_season_average,HOME_TEAM_REDCARDS_season_average,HOME_TEAM_OFFSIDES_season_average,HOME_TEAM_ATTACKS_season_average,HOME_TEAM_PENALTIES_season_average,HOME_TEAM_SUBSTITUTIONS_season_average,HOME_TEAM_BALL_SAFE_season_average,HOME_TEAM_DANGEROUS_ATTACKS_season_average,HOME_TEAM_INJURIES_season_average,HOME_TEAM_GOALS_season_average,HOME_TEAM_GAME_WON_season_average,HOME_TEAM_GAME_DRAW_season_average,HOME_TEAM_GAME_LOST_season_average,HOME_TEAM_SHOTS_TOTAL_season_std,HOME_TEAM_SHOTS_INSIDEBOX_season_std,HOME_TEAM_SHOTS_OFF_TARGET_season_std,HOME_TEAM_SHOTS_ON_TARGET_season_std,HOME_TEAM_SHOTS_OUTSIDEBOX_season_std,HOME_TEAM_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std,HOME_TEAM_SAVES_season_std,HOME_TEAM_CORNERS_season_std,HOME_TEAM_BALL_POSSESSION_season_std,HOME_TEAM_FOULS_season_std,HOME_TEAM_YELLOWCARDS_season_std,HOME_TEAM_REDCARDS_season_std,HOME_TEAM_OFFSIDES_season_std,HOME_TEAM_ATTACKS_season_std,HOME_TEAM_PENALTIES_season_std,HOME_TEAM_SUBSTITUTIONS_season_std,HOME_TEAM_BALL_SAFE_season_std,HOME_TEAM_DANGEROUS_ATTACKS_season_std,HOME_TEAM_INJURIES_season_std,HOME_TEAM_GOALS_season_std,HOME_TEAM_SHOTS_TOTAL_5_last_match_sum,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_sum,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum,HOME_TEAM_PASSES_5_last_match_sum,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_sum,HOME_TEAM_SAVES_5_last_match_sum,HOME_TEAM_CORNERS_5_last_match_sum,HOME_TEAM_FOULS_5_last_match_sum,HOME_TEAM_YELLOWCARDS_5_last_match_sum,HOME_TEAM_REDCARDS_5_last_match_sum,HOME_TEAM_OFFSIDES_5_last_match_sum,HOME_TEAM_ATTACKS_5_last_match_sum,HOME_TEAM_PENALTIES_5_last_match_sum,HOME_TEAM_SUBSTITUTIONS_5_last_match_sum,HOME_TEAM_BALL_SAFE_5_last_match_sum,HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_sum,HOME_TEAM_INJURIES_5_last_match_sum,HOME_TEAM_GOALS_5_last_match_sum,HOME_TEAM_GAME_WON_5_last_match_sum,HOME_TEAM_GAME_DRAW_5_last_match_sum,HOME_TEAM_GAME_LOST_5_last_match_sum,HOME_TEAM_SHOTS_TOTAL_5_last_match_average,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_average,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_average,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_average,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_average,HOME_TEAM_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,HOME_TEAM_SAVES_5_last_match_average,HOME_TEAM_CORNERS_5_last_match_average,HOME_TEAM_BALL_POSSESSION_5_last_match_average,HOME_TEAM_FOULS_5_last_match_average,HOME_TEAM_YELLOWCARDS_5_last_match_average,HOME_TEAM_REDCARDS_5_last_match_average,HOME_TEAM_OFFSIDES_5_last_match_average,HOME_TEAM_ATTACKS_5_last_match_average,HOME_TEAM_PENALTI

In [3]:
# Je préfère être sûr que les indexs concordent. 

train_scores = train_scores.loc[train_data.index]

train_scores.head(5)

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
0,0,0,1
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0


In [4]:
# Préparation des données pour la prédiction. 

train_scores_1c = train_scores[['HOME_WINS', 'DRAW', 'AWAY_WINS']].idxmax(axis=1)
label_mapping = {'HOME_WINS': 0, 'DRAW': 1, 'AWAY_WINS': 2}
train_scores_1c = train_scores_1c.replace(label_mapping)

train_scores_1c.head(5)

/var/folders/5w/37p1__596xz7ylx6ltmmggph0000gn/T/ipykernel_4169/2909335398.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_scores_1c = train_scores_1c.replace(label_mapping)


ID
0    2
1    1
2    2
3    0
4    1
dtype: int64

In [5]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_scores_1c, train_size=0.8, random_state=42)

HyperOpt

In [6]:
# Je définis l'espace de recherche pour les hyperparamètres de XGBoost

space = {
    'n_estimators': hp.quniform('n_estimators', 100, 2000, 50),  
    'max_depth': hp.quniform('max_depth', 3, 30, 1),  
    'max_features': hp.choice('max_features', [None, 'sqrt', 'log2', hp.uniform('max_features_frac', 0.1, 1.0)]),
    'min_samples_split': hp.quniform('min_samples_split', 2, 50, 2),  
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 20, 1),  
    'bootstrap': hp.choice('bootstrap', [True, False]),  
    'criterion': hp.choice('criterion', ['gini', 'entropy']),  
    'class_weight': hp.choice('class_weight', [None, 'balanced', 'balanced_subsample']),  
    'max_leaf_nodes': hp.quniform('max_leaf_nodes', 10, 1000, 10),  
    'min_impurity_decrease': hp.uniform('min_impurity_decrease', 0.0, 0.5)  
}


In [7]:
# La fonction que l'on va minimiser

def objective(params):
    # Convertir certains paramètres en entiers
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])
    params['max_leaf_nodes'] = int(params['max_leaf_nodes']) if params['max_leaf_nodes'] is not None else None

    max_features = params['max_features']
    if isinstance(max_features, float):  # Si c'est une fraction, on la garde
        params['max_features'] = max_features
    elif isinstance(max_features, str):  # Sinon, on garde les chaînes comme 'sqrt', 'log2', etc.
        params['max_features'] = max_features
    else:  # Si c'est None, on ne fait rien
        params['max_features'] = None
    
    # Entraîner le modèle RandomForest avec les hyperparamètres courants
    model = RandomForestClassifier(**params, random_state=42)
    
    # Entraînement
    model.fit(x_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = model.predict(x_test)
    
    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Retourner la métrique à minimiser (1 - accuracy)
    return {'loss': 1 - accuracy, 'status': STATUS_OK}


In [8]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=200,  # Augmenter le nombre d'évaluations
            trials=trials)

print("Meilleurs hyperparamètres trouvés :", best)

100%|██████████| 200/200 [13:06:21<00:00, 235.91s/trial, best loss: 0.49695245835026414]   
Meilleurs hyperparamètres trouvés : {'bootstrap': np.int64(0), 'class_weight': np.int64(0), 'criterion': np.int64(0), 'max_depth': np.float64(26.0), 'max_features': np.int64(1), 'max_leaf_nodes': np.float64(490.0), 'min_impurity_decrease': np.float64(0.00037467912627640166), 'min_samples_leaf': np.float64(19.0), 'min_samples_split': np.float64(26.0), 'n_estimators': np.float64(1000.0)}


In [18]:
# Il faut respecter le format que requiert la RandomForest pour les hyper-paramètres. 

best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['min_samples_split'] = int(best['min_samples_split'])
best['min_samples_leaf'] = int(best['min_samples_leaf'])
best['max_leaf_nodes'] = int(best['max_leaf_nodes']) if best['max_leaf_nodes'] is not None else None
best['bootstrap'] = bool(best['bootstrap'])
class_weight_options = [None, 'balanced', 'balanced_subsample']
best['class_weight'] = class_weight_options[int(best['class_weight'])] if best['class_weight'] is not None else None
criterion_options = ['gini', 'entropy', 'log_loss']
best['criterion'] = criterion_options[int(best['criterion'])] if best['criterion'] is not None else 'gini'

In [19]:
final_model = RandomForestClassifier(
    **best,
    random_state=42
)

final_model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=26, max_features=np.int64(1),
                       max_leaf_nodes=490,
                       min_impurity_decrease=np.float64(0.00037467912627640166),
                       min_samples_leaf=19, min_samples_split=26,
                       n_estimators=1000, random_state=42)

In [20]:
y_pred_final = final_model.predict(x_test)

final_accuracy = accuracy_score(y_test, y_pred_final)
print(f"Accuracy du modèle optimisé : {final_accuracy * 100:.2f}%")

Accuracy du modèle optimisé : 44.01%


# Prédictions

In [21]:
y_sub = final_model.predict(sub_data)

In [22]:
# Reconstruction du DataFrame y_sub avec les colonnes 'HOME_WINS', 'DRAW', 'AWAY_WINS'
y_sub_df = pd.DataFrame(y_sub, columns=['PRED'])

# Transformation en one-hot encoding pour obtenir les 3 colonnes
y_sub_df['HOME_WINS'] = (y_sub_df['PRED'] == 0).astype(int)
y_sub_df['DRAW'] = (y_sub_df['PRED'] == 1).astype(int)
y_sub_df['AWAY_WINS'] = (y_sub_df['PRED'] == 2).astype(int)

# Ajout de la colonne 'ID' depuis sub_data
y_sub_df['ID'] = sub_data.index

# Suppression de la colonne 'PRED'
y_sub_df.drop('PRED', axis=1, inplace=True)

# Réorganiser les colonnes pour avoir 'ID' en premier
y_sub_df = y_sub_df[['ID', 'HOME_WINS', 'DRAW', 'AWAY_WINS']]

# Afficher les premières lignes du résultat final
y_sub_df = y_sub_df.set_index('ID')
y_sub_df.head()

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
12303.0,1,0,0
12304.0,1,0,0
12305.0,1,0,0
12306.0,1,0,0
12307.0,1,0,0


In [23]:
y_sub_df.to_csv('../data/Sub_Data/5_sub_scores.csv')